### 코드 작성일 : 24.7.29 
### 코드 목표 : 대표 메뉴 변형

In [25]:
import os
from konlpy.tag import Komoran

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(original_dir)))))
    os.chdir(original_dir)
exec(open('setup/default.py').read())
pd.set_option('display.max_rows', None) # 전체 컬럼 모두 출력


In [26]:
first_menu_list = pd.read_csv(original_dir + '/data/preproc/main/feature_lab_v1/unique_first_menu.csv')

In [27]:
# 단어 사전 만들어두기 
high_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/high_cat.csv').drop_duplicates()['high_cat'].tolist()
med_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/med_cat.csv').drop_duplicates()['med_cat'].tolist()
ingredient = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/ingredient.csv').drop_duplicates()['ingredient'].tolist()
adj_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/adj_cat.csv').drop_duplicates()['adj_cat'].tolist()

custom_nouns = high_cat+med_cat+ingredient+adj_cat
with open(original_dir+'/data/feature_lab/menu_map/entity/group/user_dic.txt', 'w', encoding='utf-8') as f:
    for noun in custom_nouns:
        f.write(f"{noun}\tNNG\n")

konlp = Komoran(userdic=original_dir+'/data/feature_lab/menu_map/entity/group/user_dic.txt')

In [28]:
def konlp_split (text) :
    res = konlp.nouns(text)
    if len(res)==0 : 
        res = konlp.morphs(text)
    return res

first_menu_list['menu_comp'] = first_menu_list['menu'].progress_apply(konlp_split)

100%|██████████| 648/648 [00:00<00:00, 3632.62it/s]


In [29]:
synonyms_df = pd.read_csv(original_dir + '/data/feature_lab/menu_map/entity/synonym.csv')
synonyms = dict(zip(synonyms_df['same'], synonyms_df['top']))

# 메뉴 이름을 교체하는 함수
def replace_with_synonym(menu_items):
    # 동의어 사전에 있으면 교체, 없으면 원래 값 유지
    return [synonyms.get(item, item) for item in menu_items]

first_menu_list['menu_comp'] = first_menu_list['menu_comp'].progress_apply(replace_with_synonym)

100%|██████████| 648/648 [00:00<00:00, 491929.23it/s]


In [30]:
import copy
# 'etc' 컬럼에 menu_comp의 깊은 복사본 할당
first_menu_list['etc'] = first_menu_list['menu_comp'].apply(copy.deepcopy)

def find_and_remove_cat(etc_list, cat_values):
    for item in reversed(etc_list):
        if item in cat_values:
            etc_list.remove(item)  # 원 리스트에서 삭제 
            return item
    return None

def find_and_remove_all_cat(etc_list, cat_values):
    matched_items = [item for item in etc_list if item in cat_values]
    etc_list[:] = [item for item in etc_list if item not in cat_values]
    return matched_items if matched_items else None


first_menu_list['high_cat'] = first_menu_list['etc'].apply(lambda x: find_and_remove_cat(x, high_cat) if x else None)
first_menu_list['med_cat'] = first_menu_list['etc'].apply(lambda x: find_and_remove_cat(x, med_cat) if x else None)
first_menu_list['ingredient'] = first_menu_list['etc'].apply(lambda x: find_and_remove_all_cat(x, ingredient) if x else None)


In [31]:
first_menu_list[first_menu_list.ingredient.isna()]

,menu,menu_comp,etc,high_cat,med_cat,ingredient
2,육감쫄면,"[육감, 쫄, 면]",[육감],면,쫄,None
17,잠발라야볶음밥,"[잠발라야, 볶음, 밥]",[잠발라야],밥,볶음,None
22,잔치국수,"[잔치, 국수]",[],국수,잔치,None
29,비빔막국수,"[비빔, 막, 국수]",[비빔],국수,막,None
31,분짜,[분짜],[],분짜,None,None
46,나가사키짬뽕,"[나가사키, 짬뽕]",[나가사키],짬뽕,None,None
47,반반불고기,"[반반, 불고기]",[반반],불고기,None,None
57,경양식오므라이스,"[경양식, 오므, 밥]",[경양식],밥,오므,None
59,장터국밥,"[장터, 국, 밥]",[장터],밥,국,None
74,나주곰탕,"[나주, 곰, 탕]",[나주],탕,곰,None
